In [1]:
import os
import time
from datetime import datetime, timezone, timedelta
import pandas as pd
from huggingface_hub import HfApi, HfFolder
from tqdm import tqdm


# Constants
MIN_DOWNLOADS = 100
SINCE_DATE = datetime.now(timezone.utc) - timedelta(days=365*3)  # Collect models created in the last 3 year
OUTPUT_CSV = "hf_models.csv"
OUTPUT_DIR = "model_files"  # Directory to save model files

#os.environ["GITHUB_ACCESS_TOKEN"] = GITHUB_ACCESS_TOKEN
#os.environ["HUGGINGFACE_ACCESS_TOKEN"] = HUGGINGFACE_ACCESS_TOKEN



keywords = {
  "keywords": {
    "getting-started": {
      "gh-header-keywords": [
        "getting started",
        "installation",
        "quick start",
        "quickstart",
        "setup",
        "usage",
        "example",
        "demo"
      ],
      "hf-header-keywords": [
        "how to use",
        "usage",
        "inference",
        "example usage",
        "sample code",
        "model usage",
        "usage example"
      ],
      "gh-content-keywords": [
        "install",
        "pip",
        "requirements",
        "dependency",
        "clone",
        "git clone",
        "build",
        "run",
        "execute",
        "usage",
        "command line",
        "code snippet",
        "environment",
        "virtualenv",
        "conda",
        "script",
        "examples",
        "usage instructions",
        "dependencies",
        "install instructions"
      ],
      "hf-content-keywords": [
        "import",
        "from transformers import",
        "pipeline",
        "tokenizer",
        "model",
        "generate",
        "inference",
        "code snippet",
        "PyTorch",
        "TensorFlow",
        "example",
        "Hugging Face Hub",
        "task",
        "fine-tune",
        "load model",
        "preprocess",
        "prompt",
        "output",
        "usage example",
        "inference example"
      ]
    },
    "contributing": {
      "gh-header-keywords": [
        "contributing",
        "contribution",
        "contribute",
        "pull request",
        "bug report",
        "issue"
      ],
      "hf-header-keywords": [
        "how to contribute",
        "contribute",
        "report issues",
        "feedback",
        "suggestions",
        "contact",
        "collaborate",
        "acknowledgements"
      ],
      "gh-content-keywords": [
        "fork",
        "pull request",
        "issue tracker",
        "guidelines",
        "code style",
        "testing",
        "documentation",
        "contributing guide",
        "report bugs",
        "feature requests",
        "collaboration",
        "development",
        "submit",
        "branch",
        "merge",
        "code review",
        "community",
        "issues",
        "bug reports",
        "commit"
      ],
      "hf-content-keywords": [
        "issues",
        "contact",
        "suggestions",
        "improvements",
        "collaboration",
        "email",
        "open an issue",
        "feedback",
        "bug report",
        "help",
        "reach out",
        "community",
        "discussion",
        "contribution",
        "pull request",
        "modify",
        "enhance",
        "questions",
        "support",
        "contact author"
      ]
    },
    "license": {
      "gh-header-keywords": [
        "license",
        "licence",
        "copy right"
      ],
      "hf-header-keywords": [
        "license",
        "licence",
        "copyright",
        "terms",
        "usage terms",
        "legal",
        "rights"
      ],
      "gh-content-keywords": [
        "MIT",
        "Apache",
        "GPL",
        "BSD",
        "terms",
        "conditions",
        "distribution",
        "modification",
        "use",
        "commercial use",
        "liability",
        "warranty",
        "limitations",
        "rights",
        "reproduction",
        "software license",
        "license text",
        "open source",
        "copying",
        "proprietary"
      ],
      "hf-content-keywords": [
        "MIT",
        "Apache",
        "BSD",
        "GPL",
        "terms",
        "conditions",
        "use",
        "limitations",
        "copyright",
        "redistribution",
        "open source",
        "commercial use",
        "non-commercial",
        "Creative Commons",
        "CC BY",
        "license text",
        "restrictions",
        "public domain",
        "responsibility",
        "liability"
      ]
    }
  }
}

c:\Users\Rubik\Documents\CS573_Documentation_Quality\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:



# Authentication (optional)
ACCESS_TOKEN = os.getenv("HUGGINGFACE_ACCESS_TOKEN")  # Set your Hugging Face API token as an environment variable

# Initialize API
api = HfApi()

# If you have an access token, save it
if ACCESS_TOKEN:
    HfFolder.save_token(ACCESS_TOKEN)

# Function to collect model data
def collect_model_data():
    models_data = []
    fetch_limit = 12000  # Maximum number of models to fetch
    sort_order = "downloads"  # You can sort by 'lastModified', 'downloads', 'stars', etc.

    # Note: As of my knowledge cutoff in 2021, the Hugging Face Hub API does not support filtering by creation date directly.
    # We'll fetch models and filter them manually.

    # Search models
    print("Fetching models from Hugging Face Hub...")
    models = api.list_models(
        sort=sort_order,
        direction=-1,  # Descending order
        limit=fetch_limit,
        use_auth_token=ACCESS_TOKEN,
        full=True,  # Fetch full metadata
        
    )
    
    for model in tqdm(models):
        # Convert timestamp strings to datetime objects
        if hasattr(model, 'lastModified'):
            last_modified = model.lastModified
        else:
            last_modified = datetime.now(tz=model.lastModified.tzinfo)

        # Filter models created since the specified date
        if last_modified >= SINCE_DATE:
            # Get the model's downloads (may require an authenticated request)
            downloads = model.downloads if hasattr(model, 'downloads') else 0

            if downloads >= MIN_DOWNLOADS:
                model_data = {
                    "modelId": model.modelId,
                    "likes": model.likes if hasattr(model, 'likes') else 0,
                    "modelName": model.modelId.split("/")[-1],
                    "author": model.modelId.split("/")[0] if "/" in model.modelId else None,
                    "downloads": downloads,
                    "lastModified": model.lastModified,
                    "tags": model.tags,
                    "pipeline_tag": model.pipeline_tag,
                    "sha": model.sha,
                    "private": model.private,
                    "inference": model.inference,

                }
                #randomdly print downloads
                if downloads%22 == 0:
                    print(downloads)
                models_data.append(model_data)

        # Respect rate limits
        time.sleep(0.1)

    print(f"Total models collected: {len(models_data)}")
    return pd.DataFrame(models_data)



df = collect_model_data()

df.to_csv(OUTPUT_CSV, index=False)

Fetching models from Hugging Face Hub...


45it [00:05,  9.82it/s]

6913698


66it [00:07,  9.75it/s]

4515280


...

1452
1452


11783it [20:21,  9.61it/s]

1452


12000it [20:44,  9.64it/s]


Total models collected: 11649


In [5]:
import os
import time
import shutil
import pandas as pd
OUTPUT_CSV = "hf_models.csv"
OUTPUT_DIR = "model_cards"  # Directory to save model files
ACCESS_TOKEN = os.getenv("HUGGINGFACE_ACCESS_TOKEN") 
from huggingface_hub import HfApi, HfFolder
from tqdm import tqdm


df = pd.read_csv(OUTPUT_CSV)

# Initialize API
api = HfApi()

# Optional: Function to download model files
def download_model_files(model_ids):
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    for model_id in tqdm(model_ids):

        #check if readme already exists
        if os.path.exists(f"{OUTPUT_DIR}/{model_id.replace('/', '_')}.md"):
            #print(f"Skipping {model_id} as README.md already exists")
            continue

        #print(f"Downloading files for model: {model_id}")
        #files = api.list_repo_files(repo_id=model_id, use_auth_token=ACCESS_TOKEN)

        # Create a directory for each model
        model_dir = os.path.join(OUTPUT_DIR, model_id.replace("/", "_"))
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        #downlaod it as {model_id}.md
        try:
            api.hf_hub_download(
                repo_id=model_id,
                filename='README.md',
                local_dir=model_dir,
                use_auth_token=ACCESS_TOKEN,
            )
        except Exception as e:
            #print(f"Error downloading README.md for model {model_id}: {e}")
            continue
        #move the file to OUTPUT_DIR/{model_id}.md
        model_id = model_id.replace("/", "_")
        os.rename(f"{model_dir}/README.md", f"{OUTPUT_DIR}/{model_id}.md")
        #delete folder and contents# folder may still have files
        
        shutil.rmtree(model_dir)


        # Respect rate limits
        time.sleep(0.1)
download_model_files(df.modelId.values.tolist())

100%|██████████| 11649/11649 [17:22<00:00, 11.18it/s]


In [2]:
import pandas as pd
repos_df = pd.read_csv("hf_models.csv")
repos_df.head(20)
readme_text = []

#for every repo in the csv, get the readme
for index,row in repos_df.iterrows():
    repo_full_name = row["modelId"]
    safe_name = repo_full_name.replace("/", "_")

    #open the readme file
    try:
        with open(f"model_cards/{safe_name}.md", "r", encoding="utf-8") as f:
            readme = f.read()
            readme_text.append(readme)
    except:
        readme_text.append(None)
repos_df["readme_text"] = readme_text
repos_df = repos_df.dropna(subset=["readme_text"])

from readmepp import ReadMe
predictor = ReadMe(lang='en')
predictor.model.to('cuda')

NameError: name 'ReadMe' is not defined

# Process Dependant Columns

In [4]:
import re
import numpy as np

#count the occurences of "^- " of each readme
lists_count = repos_df["readme_text"].str.count(r"^- ", flags=re.MULTILINE)
#log then normalize it
lists_count = np.log1p(1 + lists_count)
lists_count = (lists_count - lists_count.min()) / (lists_count.max() - lists_count.min())
repos_df["lists_count"] = lists_count

#count hypperlinks
hyperlinks_count = repos_df["readme_text"].str.count(r"http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", flags=re.MULTILINE)
#log then normalize it
hyperlinks_count = np.log1p(1 + hyperlinks_count)
hyperlinks_count = (hyperlinks_count - hyperlinks_count.min()) / (hyperlinks_count.max() - hyperlinks_count.min())
repos_df["hyperlinks_count"] = hyperlinks_count

#number of images ending in .png, .jpg, .jpeg, .gif (url or local path) in .md ir .rst
#[image] (https://example.com/image.png)
#image:: https://example.com/image.png
#<.*src.*=.*\.(png|jpg|jpeg|gif|svg).*>
images_count = repos_df["readme_text"].str.count(r"!\[.*\]\(.*\.(png|jpg|jpeg|gif|svg)\)", flags=re.MULTILINE) + \
            repos_df["readme_text"].str.count(r"image::.*\.(png|jpg|jpeg|gif|svg)", flags=re.MULTILINE) + \
            repos_df["readme_text"].str.count(r"<.*src.*=.*\.(png|jpg|jpeg|gif|svg).*>", flags=re.MULTILINE)
#log then normalize it
images_count = np.log1p(1 + images_count)
images_count = (images_count - images_count.min()) / (images_count.max() - images_count.min())
repos_df["images_count"] = images_count

#number of code blocks divide by 2 and round down
code_blocks_count = (repos_df["readme_text"].str.count(r"^```", flags=re.MULTILINE) // 2).astype(int)
#log then normalize it
code_blocks_count = np.log1p(1 + code_blocks_count)
code_blocks_count = (code_blocks_count - code_blocks_count.min()) / (code_blocks_count.max() - code_blocks_count.min())
repos_df["code_blocks_count"] = code_blocks_count

#number of new lines with content
content_lines_count = repos_df["readme_text"].str.count(r"^.*[^\s]", flags=re.MULTILINE)
#log then normalize it
content_lines_count = np.log1p(1 + content_lines_count)
content_lines_count = (content_lines_count - content_lines_count.min()) / (content_lines_count.max() - content_lines_count.min())
repos_df["content_lines_count"] = content_lines_count



# Flesch Reading Score
#textstat.flesch_reading_ease(test_data)
import textstat
flesch_reading_scores = repos_df["readme_text"].apply(lambda x: textstat.flesch_reading_ease(x) if x else None)
#normalize
flesch_reading_scores = (flesch_reading_scores - flesch_reading_scores.min()) / (flesch_reading_scores.max() - flesch_reading_scores.min())
repos_df["flesch_reading_scores"] = flesch_reading_scores


# number of headers
headers_count = repos_df["readme_text"].str.count(r"#+", flags=re.MULTILINE)
#log then normalize
headers_count = np.log1p(1 + headers_count)
headers_count = (headers_count - headers_count.min()) / (headers_count.max() - headers_count.min())
repos_df["headers_count"] = headers_count

In [5]:
repos_df["readmepp"] = repos_df["readme_text"].apply(lambda x: predictor.predict(x) if x else None)

In [6]:
import re
import pandas as pd

def parse_markdown_headers(text):
    '''Parse markdown text into a list of (header_level, header_text, content) tuples'''
    lines = text.split('\n')
    headers = []
    current_header = None
    current_content = []
    current_level = None
    for line in lines:
        header_match = re.match(r'^(#{1,3})\s+(.*)', line)
        if header_match:
            # Save the current content if any
            if current_header is not None:
                headers.append((current_level, current_header, '\n'.join(current_content)))
            # Start a new header
            current_level = len(header_match.group(1))
            current_header = header_match.group(2)
            current_content = []
        else:
            if current_header is not None:
                current_content.append(line)
    # Save the last header content
    if current_header is not None:
        headers.append((current_level, current_header, '\n'.join(current_content)))
    return headers

def process_readme(text, keywords_subset,content_subset, levels=(1,2,3)):
    """
    Process a single README text.
    Returns:
        header_found: True if at least one header keyword is found in headers of specified levels.
        percentage: Percentage of keywords found in content under matching headers.
    """
    headers = parse_markdown_headers(text)
    # Initialize
    header_found = False
    content_text = ''
    index = 0
    for level, header_text, content in headers:
        if level in levels:
            # Check if any of the keywords are present in the header text
            if any(kw.lower() in header_text.lower() for kw in keywords_subset):
                header_found = True
                #then collect all header with lower level and stop untill equal or higher level
                content_text += '\n' + content
                for next_level, next_header_text, next_content in headers[index+1:]:
                    if next_level <= level:
                        break
                    content_text += '\n' + next_content

        index += 1
    # If no matching headers, percentage is 0
    if not header_found or not keywords_subset:
        return header_found, 0.0
    # Now compute the percentage of keywords found in content_text
    total_keywords = len(content_subset)
    found_keywords = sum(1 for kw in content_subset if kw.lower() in content_text.lower())
    percentage = found_keywords / total_keywords
    return header_found, percentage

# Modify your main loop
readme_type = "hf"  # "gh" or "hf"
for keyword_category in keywords["keywords"]:
    print(keyword_category)

    # Initialize columns
    header_column = f"header_{keyword_category}"
    percentage_column = f"percentage_{keyword_category}"
    #repos_df[header_column] = False  # This will be the header_found boolean
    #repos_df[percentage_column] = 0.0  # This will be the keyword percentage

    keywords_subset = keywords["keywords"][keyword_category][f"{readme_type}-header-keywords"]
    content_subset = keywords["keywords"][keyword_category][f"{readme_type}-content-keywords"]

    # Apply the function to each row
    def process_row(row):
        text = row['readme_text']
        header_found, percentage = process_readme(text, keywords_subset, content_subset)
        return pd.Series({header_column: header_found, percentage_column: percentage})

    repos_df[[header_column, percentage_column]] = repos_df.apply(process_row, axis=1)

getting-started
contributing
license


# Dependant Extraction

In [64]:
downloads_normalized = np.log1p(1 + repos_df["downloads"])
repos_df["downloads_normalized"] = (downloads_normalized - downloads_normalized.min()) / (downloads_normalized.max() - downloads_normalized.min())

likes_normalized = np.log1p(1 + repos_df["likes"])
repos_df["likes_normalized"] = (likes_normalized - likes_normalized.min()) / (likes_normalized.max() - likes_normalized.min())

heurstics = downloads_normalized/likes_normalized
heurstics = (heurstics - heurstics.min()) / (heurstics.max() - heurstics.min())
repos_df['onboarding_normalized'] = heurstics


repos_df[['readmepp_normalized']] = (repos_df[['readmepp']] - 1)/5

# Get Statistics

In [65]:
#Independant Columns normalized between 0 and 1
count_independent_columns = ["lists_count", "hyperlinks_count", "images_count", "code_blocks_count", "content_lines_count", "headers_count"]
#score Indecpend column
score_independent_columns = ["flesch_reading_scores", "readmepp_normalized"]
#keyword independant columns#binary 0 or 1
keyword_independent_columns = ["header_getting-started", "header_contributing", "header_license"]
#
dependant_columns = ["likes_normalized","downloads_normalized","onboarding_normalized"]
content_indenpendant_columns = ["percentage_contributing", "percentage_getting-started", "percentage_license"]


independent_columns = count_independent_columns + score_independent_columns + keyword_independent_columns



score_dict={"counts":{ column:{} for column in count_independent_columns}
            ,
       "keyword":{ column:{} for column in keyword_independent_columns},
       "content": { column:{} for column in content_indenpendant_columns},
       "scores":{ column:{} for column in score_independent_columns}
       }


repos_df = repos_df.dropna(subset=independent_columns + dependant_columns)
# Pearson correlation
from sklearn.feature_selection import r_regression

for dependant in dependant_columns:
    score = r_regression(repos_df[independent_columns].values, repos_df[dependant].values)

    for i, column in enumerate(independent_columns):
        print(f"Pearson correlation between {column} and {dependant}: {round(score[i],3)}")

        if column in count_independent_columns:
            score_dict["counts"][column][dependant] = round(score[i],3)
        elif column in keyword_independent_columns:
            score_dict["keyword"][column][dependant] = round(score[i],3)
        elif column in score_independent_columns:
            score_dict["scores"][column][dependant] = round(score[i],3)

for dependant in dependant_columns:
    for i in range(len(content_indenpendant_columns)):
        content_col = content_indenpendant_columns[i]
        keyword_col = keyword_independent_columns[i]
        repos_df_sub = repos_df[repos_df[keyword_col] == True]
        
        score = r_regression(repos_df_sub[[content_col]].values, repos_df_sub[dependant].values)
        print(f"Pearson correlation between {content_col} and {dependant}: {round(score[0],3)}")
        score_dict["content"][content_col][dependant] = round(score[0],3)

Pearson correlation between lists_count and likes_normalized: 0.121
Pearson correlation between hyperlinks_count and likes_normalized: 0.278
Pearson correlation between images_count and likes_normalized: 0.155
Pearson correlation between code_blocks_count and likes_normalized: 0.282
Pearson correlation between content_lines_count and likes_normalized: 0.307
Pearson correlation between headers_count and likes_normalized: 0.308
Pearson correlation between flesch_reading_scores and likes_normalized: 0.143
Pearson correlation between readmepp_normalized and likes_normalized: 0.14
Pearson correlation between header_getting-started and likes_normalized: 0.138
Pearson correlation between header_contributing and likes_normalized: 0.117
Pearson correlation between header_license and likes_normalized: 0.175
Pearson correlation between lists_count and downloads_normalized: 0.066
Pearson correlation between hyperlinks_count and downloads_normalized: 0.108
Pearson correlation between images_count a

In [78]:
# convert to pands dataframe with mulit index
# rows is the score type
# columns is the feature (with subcolumns for each method)

score_df = pd.DataFrame()
for score_type, score_data in score_dict.items():
    df_sub = pd.DataFrame(score_data).T
    df_sub = df_sub.reset_index()
    df_sub.insert(0, "score_type", score_type)
    
    score_df = pd.concat([score_df, df_sub], axis=0)

sort_array = [2] * len(content_indenpendant_columns) + [0] * len(count_independent_columns) + [1] * len(keyword_independent_columns) + [3] * len(score_independent_columns)
df_group = score_df.groupby(["score_type",'index']).mean()
df_group["sort"] = sort_array
df_group = df_group.sort_values(by=["sort"]).drop(columns="sort")
df_group

likes_normalized  downloads_normalized  \
score_type index                                                                
counts     code_blocks_count                      0.282                 0.120   
           content_lines_count                    0.307                 0.137   
           headers_count                          0.308                 0.135   
           hyperlinks_count                       0.278                 0.108   
           images_count                           0.155                -0.060   
           lists_count                            0.121                 0.066   
keyword    header_contributing                    0.117                -0.026   
           header_getting-started                 0.138                 0.107   
           header_license                         0.175                 0.011   
content    percentage_contributing                0.117                -0.059   
           percentage_getting-started             0.107                 0.035   
           percentage_license                    -0.015                -0.048   
scores     flesch_reading_scores                  0.143                 0.062   
           readmepp_normalized                    0.140                 0.026   

                                       onboarding_normalized  
score_type index                                              
counts     code_blocks_count                          -0.244  
           content_lines_count                        -0.265  
           headers_count                              -0.253  
           hyperlinks_count                           -0.237  
           images_count                               -0.158  
           lists_count                                -0.086  
keyword    header_contributing                        -0.123  
           header_getting-started                     -0.112  
           header_license                             -0.159  
content    percentage_contributing                    -0.154  
           percentage_getting-started                 -0.126  
           percentage_license                         -0.031  
scores     flesch_reading_scores                      -0.127  
           readmepp_normalized                        -0.167

In [73]:
sort_array

[2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 3, 3]

In [26]:
repos_df

,modelId,likes,modelName,author,downloads,lastModified,tags,pipeline_tag,sha,private,...,headers_count,header_getting-started,percentage_getting-started,header_contributing,percentage_contributing,header_license,percentage_license,downloads_normalized,likes_normalized,popularity_heurstic
0,sentence-transformers/all-mpnet-base-v2,891,all-mpnet-base-v2,sentence-transformers,436946085,2024-11-05 15:25:48+00:00,"['sentence-transformers', 'pytorch', 'onnx', '...",sentence-similarity,9a3225965996d404b775526de6dbfe85d3368642,False,...,0.496783,True,0.3,False,0.0,False,0.00,1.000000,0.753362,1.000000
1,sentence-transformers/all-MiniLM-L6-v2,2562,all-MiniLM-L6-v2,sentence-transformers,95520454,2024-11-01 10:26:30+00:00,"['sentence-transformers', 'pytorch', 'tf', 'ru...",sentence-similarity,fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9,False,...,0.496783,True,0.3,False,0.0,False,0.00,0.879567,0.883593,0.975130
2,nesaorg/benchmark_v0,1,benchmark_v0,nesaorg,73440489,2024-08-19 18:24:49+00:00,"['safetensors', 'model_hub_mixin', 'pytorch_mo...",NaN,7e0c8823b673ea012ce4c0f9868802648a12305d,False,...,0.082473,False,0.0,False,0.0,False,0.00,0.858746,0.050064,0.600602
3,FacebookAI/xlm-roberta-large,349,xlm-roberta-large,FacebookAI,68346199,2024-02-19 12:48:30+00:00,"['transformers', 'pytorch', 'tf', 'jax', 'onnx...",fill-mask,c23d21b0620b635a76227c604d44e43a9f0ee389,False,...,0.305935,True,0.3,False,0.0,False,0.00,0.853052,0.638063,0.851065
4,google-bert/bert-base-uncased,1914,bert-base-uncased,google-bert,68302750,2024-02-19 11:06:12+00:00,"['transformers', 'pytorch', 'tf', 'jax', 'rust...",fill-mask,86b5e0934494bd15c9632b12f734a8a67f723594,False,...,0.409839,True,0.4,False,0.0,False,0.00,0.853001,0.847622,0.941663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11643,Lajonbot/Llama-2-7b-chat-hf-instruct-pl-lora_u...,1,Llama-2-7b-chat-hf-instruct-pl-lora_unload,Lajonbot,1435,2023-07-28 07:31:07+00:00,"['transformers', 'pytorch', 'llama', 'text-gen...",text-generation,f838fda8d2b97effae1e8af4dbb6217eab14fb7e,False,...,0.000000,False,0.0,False,0.0,False,0.00,0.000000,0.050064,0.021652
11644,jondurbin/airoboros-l2-7b-2.1,6,airoboros-l2-7b-2.1,jondurbin,1435,2023-09-08 09:24:46+00:00,"['transformers', 'pytorch', 'llama', 'text-gen...",text-generation,bc5737f0e7636fae2b22c5ccdee96ec5d5e1e1e4,False,...,0.380732,True,0.3,True,0.1,True,0.25,0.000000,0.171170,0.074029
11645,CHIH-HUNG/llama-2-13b-FINETUNE1_17w,0,llama-2-13b-FINETUNE1_17w,CHIH-HUNG,1435,2023-09-13 17:41:38+00:00,"['transformers', 'pytorch', 'llama', 'text-gen...",text-generation,b499f140dc69506035dcb8212c66fcedff933d29,False,...,0.346753,False,0.0,False,0.0,False,0.00,0.000000,0.000000,0.000000
11646,CHIH-HUNG/llama-2-13b-FINETUNE1_17w-r4,0,llama-2-13b-FINETUNE1_17w-r4,CHIH-HUNG,1435,2023-09-12 12:33:46+00:00,"['transformers', 'pytorch', 'llama', 'text-gen...",text-generation,bae66e90d7e30e4d6f8cf7cd460f52b993d49244,False,...,0.346753,False,0.0,False,0.0,False,0.00,0.000000,0.000000,0.000000


In [27]:
print(f"getting-started: {repos_df['header_getting-started'].sum()}")
print(f"contributing: {repos_df['header_contributing'].sum()}")
print(f"license: {repos_df['header_license'].sum()}")

getting-started: 4190
contributing: 1453
license: 1071
